# Fisherman Game - Reinforcement learning model
Natalia Vélez, September 1, 2016

Loading dependencies:

In [2]:
import numpy as np
import numpy.random as rand
import itertools
import time
import pandas as pd
import json

## Helper functions & utilities

First, we'll define some helper functions. The functions defined in the chunk below allow us to:

1. Enumerate all of the possible actions, for some number of fishermen (`actionSpace`)
2. Compute the payoff for a particular configuration of actions (`getReward`)
3. Compute the expected reward of all possible actions (`getAllRewards` => `getExpectedRewards`)
4. Convert expected rewards into probabilities, using a softmax rule (`softMax`)
5. Marginalize over the probability distribution provided by `softMax`, to update our beliefs about each agents' actions (`marginalize`)—this will be particularly important for recursive reasoning

In [3]:
def actionSpace(n):
    return list(itertools.product(['tree', 'fish'], repeat = n))

def getReward(state, actions):
    fishermen = zip(state['strengths'], actions)
    
    fish_caught = sum([f[0] for f in fishermen if f[1] == 'fish'])
    trees_cleared = sum([f[0] for f in fishermen if f[1] == 'tree'])
    
    return fish_caught if trees_cleared >= state['trees'] else 0

def getAllRewards(state):
    '''
    NB: Because players have to try again if they reach a non-optimal solution, all payoffs are either optimal or 0
    '''
    
    # Get all possible actions
    n = len(state['strengths'])
    allActions = actionSpace(n)
    
    # Get rewards for each action
    allRewards = [getReward(state, a) for a in allActions]
    allRewards = [r if r == max(allRewards) else 0 for r in allRewards]
    
    # Return all action-payoff pairings
    return allRewards

def getExpectedReward(state, beliefs):
    # Get all possible actions and associated rewards
    n = len(state['strengths'])
    allActions = actionSpace(n)
    allRewards = getAllRewards(state)
    
    # Get probability of each set of actions in actionSpace
    singleAction = lambda a, p: p if a == 'tree' else 1-p
    actionProb = lambda actions, p: np.prod([singleAction(a, p) for a,p in zip(actions, beliefs)])
    allProbs = [actionProb(actions, beliefs) for actions in allActions]
    
    return zip(allActions, np.multiply(allRewards, allProbs))

def softMax(expectedRewards, temp):
    actions, utilities = zip(*expectedRewards)
    weightedUtilities = np.multiply(utilities, temp)
    return zip(actions, weightedUtilities/np.sum(weightedUtilities))

def marginalize(beliefs, actionProbs):
    marginals = []
    n = len(actionProbs[0][0])
    
    for agent in range(n):
        # 'tree_worlds' are all possible scenarios in which the agent chooses to clear trees, etc.
        tree_worlds = np.multiply(beliefs[agent], [p for a, p in actionProbs if a[agent] == 'tree'])
        fish_worlds = np.multiply(1-beliefs[agent], [p for a, p in actionProbs if a[agent] == 'fish'])
        
        marginals.append(sum(tree_worlds)/sum(tree_worlds + fish_worlds))
    
    return marginals

## Action selection

Now that the foundations for our model are set up, let's make some predictions! The methods below:

1. Compute the expected reward and softmax action probabilities of each permutation of actions at depth k (`actionPrediction`)
2. Use the output of `actionPrediction` to sample a single agent's actions (`agentModel`)
3. Sample actions from all agents (`combinedModel`)—this will be particularly important to run simulations, below, as we assume that each agent is choosing their actions independently, based on their beliefs about the group's likely actions.
4. Update the groups' beliefs based on the outcome of a turn, i.e, the output of `combinedModel` (`updateBeliefs`)—this part of the code is the most likely to change

In [4]:
# Returns probability of all combinations of actions
def actionPrediction(state, beliefs, depth, temp):
    # Get the probability of each outcome
    expectedRewards = getExpectedReward(state, beliefs)
    actionProbs = softMax(expectedRewards, temp)
    
    # Reason recursively about other agents, and get probability of outcomes
    if depth > 0:
        beliefs = marginalize(beliefs, actionProbs)
        actionProbs = actionPrediction(state, beliefs, depth-1, temp)
    
    return actionProbs

# Samples the actions of a single agent
def agentModel(state, beliefs, depth = 2, agent = 0, temp = 1):
    actionProbs = actionPrediction(state, beliefs, depth, temp)
    
    # Sample agent's action
    actions, probs = zip(*actionProbs)
    prediction = rand.choice(range(len(actions)), p = probs)
    
    return actions[prediction][agent]

# Samples the actions of all agents in a single turn (useful for simulation)
def combinedModel(state, beliefs, depth = 2, temp = 1):
    n = len(beliefs)
    return [agentModel(state, beliefs, depth, agent, temp) for agent in range(n)]

def updateBeliefs(state, beliefs, actions, depth, temp, alpha):
    actionProbs = actionPrediction(state, beliefs, depth, temp)
    expectation = marginalize(beliefs, actionProbs)
    
    actionsToBinary = [1 if a == 'tree' else 0 for a in actions]
    
    predictionErr = np.multiply(alpha, np.subtract(actionsToBinary, expectation))
    
    return np.add(beliefs, predictionErr)

## Putting it all together

### Counterbalancing

As a first pass, we'll compare model performance to human performance in the simplest possible way—by making the model do the same task! The code below creates a set of test scenarios to use on the model. These are structured in the same way the task itself was structured. 

In particular, there are three possible types of scenarios: scenarios with three solutions (such as `{'strengths': [1, 1, 1], 'trees': 1}`), scenarios with two solutions (e.g., `{'strengths': [1, 2, 1], 'trees': 3`), and scenarios with one solution (e.g, `{'strengths': [1, 3, 3], 'trees': 1}`). In each run of the task, the model will be tested on 8 scenarios from each category. The scenarios are ordered in blocks of 3 scenarios, such that each block has scenarios from all categories in a random order (e.g., (A B C) (C A B) (B C A) etc.).

But it's probably simpler to see the output of the function, below!

In [5]:
def counterbalancer(n):
    # Get all possible states
    allTrees = range(1,n+1)
    allStrengths = list(itertools.product(allTrees, repeat = 3))
    allStates = list(itertools.product(allStrengths, allTrees))
    allStates = [{'strengths': s, 'trees': t} for s, t in allStates] # Convert to dictionary

    for state in allStates:
        payoffs = getAllRewards(state)
        state['n_solutions'] = np.count_nonzero(payoffs)

    # Remove states with no solutions
    allStates = [state for state in allStates if state['n_solutions'] > 0]

    # Group all states according to the number of possible solutions
    state_groups = np.unique([state['n_solutions'] for state in allStates])
    statesByGroup = [[state for state in allStates if state['n_solutions'] == group] for group in state_groups]

    # Sample 8 states from each group
    randomSubset = [rand.choice(group, size = 8, replace = False) for group in statesByGroup]

    # Arrange into blocks and randomize within block
    trialOrder = zip(*randomSubset)
    trialOrder = [rand.permutation(block) for block in trialOrder]
    trialOrder = np.vstack(trialOrder).reshape(-1) # Flattened array
    
    # Add trial number
    for t in range(len(trialOrder)):
        trialOrder[t]['num'] = t+1
    
    return trialOrder

# Example output
counterbalancer(3)

array([{'n_solutions': 3, 'strengths': (3, 3, 3), 'num': 1, 'trees': 3},
       {'n_solutions': 1, 'strengths': (3, 3, 1), 'num': 2, 'trees': 1},
       {'n_solutions': 2, 'strengths': (3, 1, 1), 'num': 3, 'trees': 1},
       {'n_solutions': 3, 'strengths': (1, 1, 1), 'num': 4, 'trees': 2},
       {'n_solutions': 1, 'strengths': (2, 1, 3), 'num': 5, 'trees': 1},
       {'n_solutions': 2, 'strengths': (3, 2, 2), 'num': 6, 'trees': 2},
       {'n_solutions': 1, 'strengths': (2, 2, 3), 'num': 7, 'trees': 3},
       {'n_solutions': 3, 'strengths': (2, 2, 2), 'num': 8, 'trees': 3},
       {'n_solutions': 2, 'strengths': (2, 2, 1), 'num': 9, 'trees': 3},
       {'n_solutions': 3, 'strengths': (1, 1, 1), 'num': 10, 'trees': 1},
       {'n_solutions': 2, 'strengths': (3, 2, 1), 'num': 11, 'trees': 3},
       {'n_solutions': 1, 'strengths': (3, 2, 2), 'num': 12, 'trees': 3},
       {'n_solutions': 1, 'strengths': (1, 1, 3), 'num': 13, 'trees': 3},
       {'n_solutions': 3, 'strengths': (3, 3, 3

### Running the simulation

The moment we've all been waiting for! The function `simulate` puts everything together and:

1. Creates a new trial order for each simulation
2. Passes the model through each trial and receives each agent's response
3. Makes agents try again if they don't get any reward
4. Updates prior on each agent's actions
5. Passes the new priors on to the next trial
6. Returns the model's trial history

In [ ]:
def simulate(n, depth, temp, alpha):
    trialOrder = counterbalancer(n)
    beliefs = [.5]*n

    trialHistory = []
    for trial in trialOrder:
        while True:
            actions = combinedModel(trial, beliefs, depth, temp)
            payoff = getReward(trial, actions)
            beliefs = updateBeliefs(trial, beliefs, actions, depth, temp, alpha)

            trialInfo = dict(trial, **{'actions': actions, \
                                       'beliefs': list(beliefs), \
                                       'payoff': payoff, \
                                       'is_max': payoff > 0})
            trialHistory.append(trialInfo)

            if payoff > 0:
                break
    
    return trialHistory

## Analyzing simulation

Now that the simulator is ready, we're going to run many, many simulations. (This code takes a little over a minute to run.)

In [ ]:
# Model parameters
n = 3
depth = 3
temp = 2.5
alpha = 0.1

# Simulation parameters
n_iterations = 1000

# Let's go!
startTime = time.clock()
allSimulations = [simulate(n, depth, temp, alpha) for i in range(n_iterations)]
endTime = time.clock()
print 'Simulation completed in %0.2f seconds' % (endTime - startTime)

Now we'll save the output of the simulation as a CSV file and analyze it in R.

In [ ]:
simulationData = []

# Change into dataset-friendly format (e.g., by splitting lists into multiple columns)
for iteration in range(n_iterations):
    for trial in range(len(allSimulations[iteration])):
        trialInfo = allSimulations[iteration][trial]
        newData = dict(trialInfo, **{'iteration': iteration+1, \
                                     'p1_prior': trialInfo['beliefs'][0], \
                                     'p2_prior': trialInfo['beliefs'][1], \
                                     'p3_prior': trialInfo['beliefs'][2], \
                                     'p1_strength': trialInfo['strengths'][0], \
                                     'p2_strength': trialInfo['strengths'][1], \
                                     'p3_strength': trialInfo['strengths'][2], \
                                     'p1_action': trialInfo['actions'][0], \
                                     'p2_action': trialInfo['actions'][1], \
                                     'p3_action': trialInfo['actions'][2], \
                                    })
        
        # Remove list variables
        del newData['beliefs']
        del newData['actions']
        del newData['strengths']
        
        simulationData.append(newData)

# Change list of dictionaries into a dictionary of lists, and then into a pandas dataframe
simulationData = {key:[item[key] for item in simulationData] for key in simulationData[0].keys()}
simulationData = pd.DataFrame(simulationData)

# Save simulation parameters to separate file
timestamp = time.strftime("%Y%m%d_%H%M%S")
simulationParams = {'n': n, 'depth': depth, 'temp': temp, 'alpha': alpha, 'n_iterations': n_iterations}

simulationData.to_csv('simulation_results_%s.csv' % (timestamp))

with open('simulation_params_%s.json' % (timestamp), 'w') as f:
    json.dump(simulationParams, f)

Debugging: testing this model's predictions against Kelsey's original fisherman model

In [13]:
import action_prediction_clean as ka

R = ka.getRewards_multi([1,1,1],1)
print ka.QRE_multi(R,2,2.5, np.array([.55, .5, .5]))

state = {'trees': 1, 'strengths': [1, 1, 1]}
beliefs = [.55, .5, .5]
probs = actionPrediction(state, beliefs, 2, 2.5)
print marginalize(beliefs, probs)

0.749912724973
[0.69126374554776404, 0.17655879940547115, 0.17655879940547115]
